## Paso 1: Instalar dependencias

In [1]:
!pip install tensorflow tensorflow-datasets matplotlib opencv-python pillow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


## Paso 2: Importar librerías

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image

## Paso 3: Descargar dataset de perros y gatos

### ⚠️ SOLUCIÓN ALTERNATIVA si hay problemas con el dataset

Si el paso anterior falla, puedes usar esta solución alternativa más simple:

In [15]:
# ALTERNATIVA: Descargar dataset manualmente desde Kaggle
# Si el método anterior no funciona, ejecuta esto:

# 1. Ve a: https://www.microsoft.com/en-us/download/details.aspx?id=54765
# 2. Descarga el archivo ZIP
# 3. Extráelo en una carpeta llamada 'PetImages' en la misma ubicación de este notebook
# 4. La estructura debe ser: PetImages/Cat/ y PetImages/Dog/

# Función para cargar desde carpetas locales
def cargar_desde_carpetas_locales():
    """Carga imágenes si tienes las carpetas PetImages/Cat y PetImages/Dog"""
    datos_local = []
    
    carpetas = [
        ('PetImages/Cat', 0),  # Gatos = 0
        ('PetImages/Dog', 1)   # Perros = 1
    ]
    
    for carpeta, etiqueta in carpetas:
        if not os.path.exists(carpeta):
            continue
            
        archivos = [f for f in os.listdir(carpeta) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"Cargando {len(archivos)} imágenes de {carpeta}...")
        
        for i, archivo in enumerate(archivos[:5000]):  # Máximo 5000 por clase
            try:
                ruta = os.path.join(carpeta, archivo)
                img = cv2.imread(ruta)
                if img is not None:
                    datos_local.append((img, etiqueta))
            except:
                continue
                
            if (i + 1) % 500 == 0:
                print(f"  Procesadas {i + 1}...")
    
    return datos_local

# Intentar cargar desde carpetas locales si existen
if os.path.exists('PetImages'):
    print("📁 Encontradas carpetas locales de PetImages")
    datos_local = cargar_desde_carpetas_locales()
    if len(datos_local) > 0:
        # Convertir a formato similar a tensorflow_datasets
        print(f"✓ Cargadas {len(datos_local)} imágenes localmente")
else:
    print("ℹ️ No se encontraron carpetas locales (esto es normal si es la primera vez)")

ℹ️ No se encontraron carpetas locales (esto es normal si es la primera vez)


In [8]:
# Corrección temporal para el dataset
setattr(tfds.image_classification.cats_vs_dogs, '_URL',
        "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

# Descargar el set de datos
datos, metadatos = tfds.load('cats_vs_dogs', as_supervised=True, with_info=True)
print(f"Total de imágenes: {metadatos.splits['train'].num_examples}")

Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 144.19 url/s]


KeyError: "There is no item named 'PetImages\\\\Cat\\\\0.jpg' in the archive"

## Paso 4: IMPORTANTE - Crear carpeta con tus fotos

**INSTRUCCIONES:**
1. Crea una carpeta llamada `mis_fotos` en la misma ubicación que este notebook
2. Agrega al menos 50-100 fotos tuyas (diferentes ángulos, iluminación, fondos)
3. Las fotos pueden ser JPG, PNG, etc.
4. Pueden ser selfies, fotos de tu cara en diferentes situaciones

Ruta esperada: `clasificador-perros-gatos-main/mis_fotos/`

In [ ]:
# Definir el tamaño de imagen
TAMANO_IMG = 100

# Verificar que existe la carpeta con tus fotos
CARPETA_MIS_FOTOS = 'mis_fotos'

if not os.path.exists(CARPETA_MIS_FOTOS):
    print(f"⚠️ ERROR: No existe la carpeta '{CARPETA_MIS_FOTOS}'")
    print(f"Por favor, crea la carpeta y agrega tus fotos allí.")
else:
    num_fotos = len([f for f in os.listdir(CARPETA_MIS_FOTOS) 
                     if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
    print(f"✓ Carpeta encontrada con {num_fotos} fotos")
    if num_fotos < 50:
        print(f"⚠️ ADVERTENCIA: Se recomienda al menos 50 fotos. Tienes {num_fotos}")

## Paso 5: Cargar y procesar tus fotos

In [ ]:
def cargar_mis_fotos(carpeta, tamano_img=100):
    """Carga y procesa las fotos de la carpeta especificada"""
    imagenes = []
    extensiones_validas = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')
    
    archivos = [f for f in os.listdir(carpeta) if f.lower().endswith(extensiones_validas)]
    
    print(f"Procesando {len(archivos)} imágenes...")
    
    for i, archivo in enumerate(archivos):
        try:
            ruta = os.path.join(carpeta, archivo)
            # Leer imagen
            imagen = cv2.imread(ruta)
            if imagen is None:
                print(f"⚠️ No se pudo leer: {archivo}")
                continue
            
            # Redimensionar
            imagen = cv2.resize(imagen, (tamano_img, tamano_img))
            # Convertir a escala de grises
            imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
            # Reshape para el formato del modelo
            imagen = imagen.reshape(tamano_img, tamano_img, 1)
            
            imagenes.append(imagen)
            
            if (i + 1) % 20 == 0:
                print(f"  Procesadas {i + 1}/{len(archivos)}")
        except Exception as e:
            print(f"⚠️ Error al procesar {archivo}: {e}")
    
    print(f"✓ Total procesadas: {len(imagenes)}")
    return np.array(imagenes)

# Cargar tus fotos
mis_imagenes = cargar_mis_fotos(CARPETA_MIS_FOTOS, TAMANO_IMG)
print(f"\nShape de tus imágenes: {mis_imagenes.shape}")

## Paso 6: Visualizar algunas de tus fotos procesadas

In [ ]:
# Mostrar algunas de tus fotos procesadas
plt.figure(figsize=(15, 5))
num_mostrar = min(10, len(mis_imagenes))
for i in range(num_mostrar):
    plt.subplot(2, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(mis_imagenes[i].reshape(TAMANO_IMG, TAMANO_IMG), cmap='gray')
    plt.title("Tú")
plt.tight_layout()
plt.show()

## Paso 7: Procesar dataset de perros y gatos

In [ ]:
# Procesar todas las imágenes del dataset
datos_entrenamiento = []

print("Procesando imágenes de perros y gatos...")
for i, (imagen, etiqueta) in enumerate(datos['train']):
    # Redimensionar y convertir a escala de grises
    imagen = cv2.resize(imagen.numpy(), (TAMANO_IMG, TAMANO_IMG))
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    imagen = imagen.reshape(TAMANO_IMG, TAMANO_IMG, 1)
    
    # Guardar imagen con su etiqueta original (0=gato, 1=perro)
    datos_entrenamiento.append([imagen, etiqueta.numpy()])
    
    if (i + 1) % 2000 == 0:
        print(f"  Procesadas {i + 1} imágenes...")

print(f"✓ Total de perros y gatos: {len(datos_entrenamiento)}")

## Paso 8: Combinar todos los datos (gatos, perros y tú)

In [ ]:
# Agregar tus fotos al dataset con etiqueta 2
for imagen in mis_imagenes:
    datos_entrenamiento.append([imagen, 2])  # Etiqueta 2 = Tú

print(f"Total de imágenes en el dataset combinado: {len(datos_entrenamiento)}")

# Mezclar los datos aleatoriamente
np.random.shuffle(datos_entrenamiento)
print("✓ Datos mezclados aleatoriamente")

## Paso 9: Separar imágenes y etiquetas

In [ ]:
# Preparar variables X (entradas) y y (etiquetas)
X = []
y = []

for imagen, etiqueta in datos_entrenamiento:
    X.append(imagen)
    y.append(etiqueta)

# Convertir a arrays numpy
X = np.array(X).astype(float) / 255  # Normalizar
y = np.array(y)

print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"\nDistribución de clases:")
print(f"  Gatos (0): {np.sum(y == 0)}")
print(f"  Perros (1): {np.sum(y == 1)}")
print(f"  Tú (2): {np.sum(y == 2)}")

## Paso 10: Visualizar ejemplos de las 3 clases

In [ ]:
# Mostrar ejemplos de cada clase
plt.figure(figsize=(15, 5))
clases = ['Gato', 'Perro', 'Tú']

for clase_idx in range(3):
    # Encontrar índices de esta clase
    indices = np.where(y == clase_idx)[0][:5]
    
    for i, idx in enumerate(indices):
        plt.subplot(3, 5, clase_idx * 5 + i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(X[idx].reshape(TAMANO_IMG, TAMANO_IMG), cmap='gray')
        if i == 0:
            plt.ylabel(clases[clase_idx], fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## Paso 11: Crear el modelo CNN para 3 clases

In [ ]:
# Modelo CNN para 3 clases
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(250, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 clases con softmax
])

# Compilar el modelo
modelo.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Para múltiples clases
    metrics=['accuracy']
)

modelo.summary()

## Paso 12: Entrenar el modelo

In [ ]:
# Entrenar el modelo
history = modelo.fit(
    X, y,
    batch_size=32,
    validation_split=0.15,
    epochs=50,  # Puedes ajustar según necesites
    verbose=1
)

## Paso 13: Visualizar el entrenamiento

In [ ]:
# Graficar accuracy y loss
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title('Precisión del Modelo')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida del Modelo')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Paso 14: Probar el modelo con ejemplos aleatorios

In [ ]:
# Probar con imágenes aleatorias
clases_nombres = ['Gato', 'Perro', 'Tú']
indices_prueba = np.random.choice(len(X), 12, replace=False)

plt.figure(figsize=(15, 8))
for i, idx in enumerate(indices_prueba):
    # Hacer predicción
    imagen_test = X[idx].reshape(1, 100, 100, 1)
    prediccion = modelo.predict(imagen_test, verbose=0)
    clase_predicha = np.argmax(prediccion)
    confianza = prediccion[0][clase_predicha] * 100
    
    # Mostrar imagen
    plt.subplot(3, 4, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X[idx].reshape(TAMANO_IMG, TAMANO_IMG), cmap='gray')
    
    # Color verde si es correcto, rojo si no
    color = 'green' if clase_predicha == y[idx] else 'red'
    plt.title(f"Real: {clases_nombres[y[idx]]}\nPred: {clases_nombres[clase_predicha]} ({confianza:.1f}%)",
              color=color, fontsize=9)

plt.tight_layout()
plt.show()

## Paso 15: Guardar el modelo

In [ ]:
# Guardar el modelo entrenado
modelo.save('clasificador_personalizado.h5')
print("✓ Modelo guardado como 'clasificador_personalizado.h5'")

## Paso 16: Convertir a TensorFlow.js (opcional)

In [ ]:
# Instalar tensorflowjs
!pip install tensorflowjs

In [ ]:
# Crear carpeta y convertir
!mkdir modelo_web
!tensorflowjs_converter --input_format keras clasificador_personalizado.h5 modelo_web
print("✓ Modelo convertido para web en la carpeta 'modelo_web'")

## Paso 17: Función para probar con una imagen nueva

In [ ]:
def predecir_imagen(ruta_imagen):
    """Predice la clase de una imagen desde un archivo"""
    # Leer y procesar imagen
    imagen = cv2.imread(ruta_imagen)
    imagen = cv2.resize(imagen, (TAMANO_IMG, TAMANO_IMG))
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    imagen = imagen.reshape(1, TAMANO_IMG, TAMANO_IMG, 1)
    imagen = imagen.astype(float) / 255
    
    # Hacer predicción
    prediccion = modelo.predict(imagen, verbose=0)
    clase = np.argmax(prediccion)
    confianza = prediccion[0][clase] * 100
    
    clases_nombres = ['Gato', 'Perro', 'Tú']
    
    print(f"\nPredicción: {clases_nombres[clase]}")
    print(f"Confianza: {confianza:.2f}%")
    print(f"\nProbabilidades:")
    for i, prob in enumerate(prediccion[0]):
        print(f"  {clases_nombres[i]}: {prob*100:.2f}%")
    
    # Mostrar imagen
    img_mostrar = cv2.imread(ruta_imagen)
    img_mostrar = cv2.cvtColor(img_mostrar, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6, 6))
    plt.imshow(img_mostrar)
    plt.title(f"Predicción: {clases_nombres[clase]} ({confianza:.1f}%)")
    plt.axis('off')
    plt.show()

# Ejemplo de uso (descomenta y cambia la ruta):
# predecir_imagen('mis_fotos/foto1.jpg')